## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\izee\OneDrive\Desktop\Data Analysis and Visualization\Module 6 - WeatherPy with Python APIs\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,63.81,62,0,3.58,clear sky
1,1,Albany,US,42.6001,-73.9662,57.94,36,14,4.18,few clouds
2,2,Nikolskoye,RU,59.7035,30.7861,39.09,95,81,2.98,broken clouds
3,3,Keffi,NG,8.8486,7.8736,90.50,34,48,5.61,scattered clouds
4,4,Luderitz,NaN,-26.6481,15.1594,70.56,56,3,4.25,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Luderitz,NaN,-26.6481,15.1594,70.56,56,3,4.25,clear sky
18,18,Castro,BR,-24.7911,-50.0119,75.70,65,65,8.28,light rain
19,19,Balabac,PH,7.9868,117.0645,79.95,77,100,10.49,overcast clouds
22,22,Souillac,MU,-20.5167,57.5167,77.38,83,75,13.80,light rain
24,24,Pringsewu,ID,-5.3582,104.9744,73.58,95,87,3.91,overcast clouds
25,25,Puerto Ayora,EC,-0.7393,-90.3518,84.16,90,91,8.01,overcast clouds
30,30,Georgetown,MY,5.4112,100.3354,80.53,89,20,5.01,few clouds
31,31,Rikitea,PF,-23.1203,-134.9692,79.02,72,1,17.98,clear sky
36,36,Santa Isabel Do Rio Negro,BR,-0.4139,-65.0192,82.87,86,98,3.33,moderate rain
38,38,Meulaboh,ID,4.1363,96.1285,76.28,89,85,3.83,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                244
City                   244
Country                238
Lat                    244
Lng                    244
Max Temp               244
Humidity               244
Cloudiness             244
Wind Speed             244
Current Description    244
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
clean_df.count()

City_ID                238
City                   238
Country                238
Lat                    238
Lng                    238
Max Temp               238
Humidity               238
Cloudiness             238
Wind Speed             238
Current Description    238
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Castro,BR,75.70,light rain,-24.7911,-50.0119,
19,Balabac,PH,79.95,overcast clouds,7.9868,117.0645,
22,Souillac,MU,77.38,light rain,-20.5167,57.5167,
24,Pringsewu,ID,73.58,overcast clouds,-5.3582,104.9744,
25,Puerto Ayora,EC,84.16,overcast clouds,-0.7393,-90.3518,
30,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
31,Rikitea,PF,79.02,clear sky,-23.1203,-134.9692,
36,Santa Isabel Do Rio Negro,BR,82.87,moderate rain,-0.4139,-65.0192,
38,Meulaboh,ID,76.28,overcast clouds,4.1363,96.1285,
40,Koumac,NC,75.83,few clouds,-20.5667,164.2833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.    
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Castro,BR,75.70,light rain,-24.7911,-50.0119,CHACARA BAILLY
19,Balabac,PH,79.95,overcast clouds,7.9868,117.0645,Batalla’s Lodging House
22,Souillac,MU,77.38,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
24,Pringsewu,ID,73.58,overcast clouds,-5.3582,104.9744,Urban Style by Front One
25,Puerto Ayora,EC,84.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
30,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
31,Rikitea,PF,79.02,clear sky,-23.1203,-134.9692,People ThankYou
36,Santa Isabel Do Rio Negro,BR,82.87,moderate rain,-0.4139,-65.0192,Hotel GSA
38,Meulaboh,ID,76.28,overcast clouds,4.1363,96.1285,Kost Pria
40,Koumac,NC,75.83,few clouds,-20.5667,164.2833,Le Monitel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = (r"C:\Users\izee\OneDrive\Desktop\Data Analysis and Visualization\Module 6 - WeatherPy with Python APIs\World_Weather_Analysis\Vacation_Search\WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))